#Projeto para atendimento e suporte

In [78]:
from langchain_groq import ChatGroq
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
import os
import getpass

In [137]:
!pip show langchain
!pip show langchain-core

Name: langchain
Version: 1.2.0
Summary: Building applications with LLMs through composability
Home-page: https://docs.langchain.com/
Author: 
Author-email: 
License: MIT
Location: /usr/local/lib/python3.12/dist-packages
Requires: langchain-core, langgraph, pydantic
Required-by: 
Name: langchain-core
Version: 1.2.5
Summary: Building applications with LLMs through composability
Home-page: https://docs.langchain.com/
Author: 
Author-email: 
License: MIT
Location: /usr/local/lib/python3.12/dist-packages
Requires: jsonpatch, langsmith, packaging, pydantic, pyyaml, tenacity, typing-extensions, uuid-utils
Required-by: langchain, langchain-classic, langchain-community, langchain-groq, langchain-huggingface, langchain-text-splitters, langgraph, langgraph-checkpoint, langgraph-prebuilt


In [ ]:
os.environ["GROQ_API_KEY"] = getpass.getpass("Digite a chave da API do GROQ: ")

Digite a chave da API do GROQ: ··········


In [79]:
def loadllm(id_model, temperature):
  llm = ChatGroq(
      model = id_model,
      temperature = temperature,
      max_tokens = None,
      timeout = None,
      max_retries=2,
  )
  return llm

In [80]:
id_model = "llama-3.3-70b-versatile" #@param {type: "string"}
temperature = 0.7 # @param {type: "slider", min:0.1, max:1}

llm = loadllm(id_model, temperature)

In [81]:
prompt = "Desejo recuperar minha senha do sistema "  # @param {type: "string"}

template = ChatPromptTemplate.from_messages([
    ("system", "você é um assistente virtual prestativo respondendo perguntas gerais"),
    ("human", "{prompt}")
])

chain = template | llm | StrOutputParser()
res = chain.invoke({"prompt": prompt})
res

'Entendo que você precisa recuperar sua senha do sistema. Para ajudá-lo, preciso saber um pouco mais sobre o sistema em questão e as etapas que você já tentou. Aqui estão algumas dicas gerais que podem ser úteis:\n\n1. **Verifique se você tem acesso a um e-mail de recuperação**: Muitos sistemas permitem que você forneça um endereço de e-mail de recuperação quando cria sua conta. Se esse for o caso, você pode usar essa opção para receber um link ou código de recuperação de senha.\n\n2. **Tente a opção "Esqueci minha senha"**: A maioria dos sistemas tem uma opção "Esqueci minha senha" na página de login. Clique nessa opção e siga as instruções para iniciar o processo de recuperação de senha.\n\n3. **Verifique sua caixa de spam**: Se você solicitou um e-mail de recuperação de senha, certifique-se de verificar sua caixa de spam, pois o e-mail pode ter sido enviado para lá em vez da sua caixa de entrada.\n\n4. **Contate o suporte técnico**: Se nenhuma das opções acima funcionar, pode ser ne

In [100]:
def show(res):
  from IPython.display import Markdown
  if "</think>" in res:
    res = res.split("</think>")[-1].strip()
  else:
    res = res.strip()
    display(Markdown(res))
show(res)

Para alterar sua senha, existem duas opções dependendo de como você está acessando a plataforma: pelo aplicativo ou pelo site.

**Alterando a senha pelo aplicativo:**

1. Abra o aplicativo.
2. Clique no menu "Minha conta".
3. Selecione a opção "Alterar senha".

**Alterando a senha pelo site:**

1. Acesse o site.
2. Clique em "Configurações" no menu do topo.
3. Selecione "Minha conta".
4. Em seguida, escolha a opção "Alterar senha".

Em ambos os casos, após selecionar "Alterar senha", você provavelmente será solicitado a digitar a senha atual e, em seguida, criar e confirmar a nova senha. Certifique-se de que a nova senha atende aos requisitos de segurança, como ter uma combinação de letras maiúsculas e minúsculas, números e símbolos. Após inserir as informações necessárias, confirme as alterações para salvar a nova senha.

In [101]:
context = """
Para alterar uma senha no aplicativo, clique no menu 'Minha conta' e selecione 'Alterar senha'.
Para alterar a senha pelo site, acesse 'Configurações' no menu do topo. Em seguida, selecione 'Minha conta' e 'Alterar senha'.
"""

prompt = f"""
Como alterar minha senha?

Contexto: {context}
"""

In [102]:
prompt

"\nComo alterar minha senha?\n\nContexto: \nPara alterar uma senha no aplicativo, clique no menu 'Minha conta' e selecione 'Alterar senha'.\nPara alterar a senha pelo site, acesse 'Configurações' no menu do topo. Em seguida, selecione 'Minha conta' e 'Alterar senha'.\n\n"

In [103]:
template_rag = """
Pergunta:: {input}
Contexto:: {context}
"""

In [104]:
from langchain_core.prompts import PromptTemplate

prompt_rag = PromptTemplate.from_template(template_rag)
print(prompt_rag)

input_variables=['context', 'input'] input_types={} partial_variables={} template='\nPergunta:: {input}\nContexto:: {context}\n'


In [105]:
chain_rag = prompt_rag | llm | StrOutputParser()

input = "como alterar minhar senha?"

res = chain_rag.invoke({"context": context, "input": input})

show(res)

Para alterar sua senha, você pode seguir os passos abaixo, dependendo de onde deseja fazer a alteração:

**No Aplicativo:**
1. Abra o aplicativo.
2. Clique no menu "Minha conta".
3. Selecione "Alterar senha".

**No Site:**
1. Acesse o site.
2. Clique em "Configurações" no menu do topo.
3. Selecione "Minha conta".
4. Clique em "Alterar senha".

Em ambos os casos, após selecionar "Alterar senha", você será solicitado a digitar sua senha atual e, em seguida, criar uma nova senha. Certifique-se de que a nova senha atenda aos requisitos de segurança, como ter um comprimento mínimo e incluir uma combinação de letras e números. Após confirmar a nova senha, sua alteração será efetuada.

In [106]:
#Indexação de documentos

In [110]:
from google.colab import files
uploaded = files.upload()
files_path = list(uploaded.keys())[0]
print(f"arquivo carregado: {files_path}")

Saving manual-safebank.pdf to manual-safebank.pdf
arquivo carregado: manual-safebank.pdf


In [111]:
from pathlib import Path
from langchain_community.document_loaders import PyMuPDFLoader

file_path = "manual-safebank.pdf"

load = PyMuPDFLoader(file_path)
docs = load.load()
docs[1]

Document(metadata={'producer': 'Skia/PDF m137 Google Docs Renderer', 'creator': '', 'creationdate': '', 'source': 'manual-safebank.pdf', 'file_path': 'manual-safebank.pdf', 'total_pages': 11, 'format': 'PDF 1.4', 'title': 'manual-safebank', 'author': '', 'subject': '', 'keywords': '', 'moddate': '', 'trapped': '', 'modDate': '', 'creationDate': '', 'page': 1}, page_content='Prazo para ativação da conta  \nO processo de verificação leva até 2 dias úteis. Em casos com divergências de dados ou \nimagens ilegíveis, esse prazo pode se estender. O cliente será notificado por e-mail ou \nnotificação no app. \n \nProblemas comuns no cadastro \n●\u200b Documento vencido ou ilegível \n●\u200b Foto do documento fora do enquadramento \n●\u200b Não realização do vídeo de reconhecimento facial \n●\u200b Dados incorretos ou divergentes com a Receita Federal \n \n \n3. Acesso à Conta \nAcesso via site - Acesse o site oficial e clique em "Entrar" no topo da página. Insira seu \nCPF e senha. Caso deseje

In [112]:
print(docs[0].page_content[:1000])

Manual de Atendimento e Uso – SafeBank 
 
1. Introdução 
Objetivo do Manual  
Este manual tem como objetivo orientar clientes e atendentes sobre o funcionamento 
completo do banco digital, abordando desde o acesso à conta, funcionalidades e serviços, 
até procedimentos de segurança, suporte e resolução de problemas. O documento também 
serve como material de consulta para dúvidas frequentes. 
Visão geral do banco digital  
Nosso banco digital oferece serviços bancários completos de forma 100% online, por meio 
do aplicativo e do site. Entre os principais serviços estão conta digital gratuita, 
transferências via Pix, pagamento de boletos, emissão de cartões, investimentos, seguros e 
muito mais, sempre com foco em praticidade, segurança e transparência. 
Público-alvo  
O banco digital é voltado para pessoas físicas a partir de 18 anos, que buscam soluções 
financeiras modernas, acessíveis e com gestão 100% digital.  
 
2. Cadastro e Abertura de Conta 
Requisitos para abertura de conta 

In [113]:
def extract_text_pdf(file_path):
  loader = PyMuPDFLoader(file_path)
  docs = loader.load()
  content = "\n".join([page.page_content for page in docs])
  return content

In [114]:
extract_text_pdf(file_path)

'Manual de Atendimento e Uso – SafeBank \n \n1. Introdução \nObjetivo do Manual  \nEste manual tem como objetivo orientar clientes e atendentes sobre o funcionamento \ncompleto do banco digital, abordando desde o acesso à conta, funcionalidades e serviços, \naté procedimentos de segurança, suporte e resolução de problemas. O documento também \nserve como material de consulta para dúvidas frequentes. \nVisão geral do banco digital  \nNosso banco digital oferece serviços bancários completos de forma 100% online, por meio \ndo aplicativo e do site. Entre os principais serviços estão conta digital gratuita, \ntransferências via Pix, pagamento de boletos, emissão de cartões, investimentos, seguros e \nmuito mais, sempre com foco em praticidade, segurança e transparência. \nPúblico-alvo  \nO banco digital é voltado para pessoas físicas a partir de 18 anos, que buscam soluções \nfinanceiras modernas, acessíveis e com gestão 100% digital.  \n \n2. Cadastro e Abertura de Conta \nRequisitos para

In [115]:
docs_path = Path("/content")
pdf_files = [f for f in docs_path.glob("*.pdf")]
print(pdf_files)

[PosixPath('/content/manual-safebank.pdf')]


In [116]:
loaded_documents = [extract_text_pdf(pdf) for pdf in pdf_files]
len(pdf_files)

1

#Divisão do texto

In [117]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

dividir_text = RecursiveCharacterTextSplitter(
    chunk_size = 500,
    chunk_overlap  = 50
)
chunks = []

for docs in loaded_documents:
  chunks.extend(dividir_text.create_documents([docs]))

  print(f"Número de chunks: {len(chunks)}")

Número de chunks: 44


In [118]:
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_community.vectorstores import FAISS

embeddings_modelo = "BAAI/bge-m3"

embeddings = HuggingFaceEmbeddings(model_name = embeddings_modelo)

input_teste = "Teste numero 01"

resultado = embeddings.embed_query(input_teste)



In [119]:
len(resultado)

1024

In [120]:
print(resultado[:10])

[-0.007981249131262302, 0.03925924375653267, -0.0458027720451355, -0.011526712216436863, 0.007480816915631294, -0.009658632799983025, 0.02111714333295822, 0.026043273508548737, -0.0025459900498390198, 0.005056781694293022]


In [121]:
chunks[0]

Document(metadata={}, page_content='Manual de Atendimento e Uso – SafeBank \n \n1. Introdução \nObjetivo do Manual  \nEste manual tem como objetivo orientar clientes e atendentes sobre o funcionamento \ncompleto do banco digital, abordando desde o acesso à conta, funcionalidades e serviços, \naté procedimentos de segurança, suporte e resolução de problemas. O documento também \nserve como material de consulta para dúvidas frequentes. \nVisão geral do banco digital')

In [122]:
chunks[1]

Document(metadata={}, page_content='Visão geral do banco digital  \nNosso banco digital oferece serviços bancários completos de forma 100% online, por meio \ndo aplicativo e do site. Entre os principais serviços estão conta digital gratuita, \ntransferências via Pix, pagamento de boletos, emissão de cartões, investimentos, seguros e \nmuito mais, sempre com foco em praticidade, segurança e transparência. \nPúblico-alvo  \nO banco digital é voltado para pessoas físicas a partir de 18 anos, que buscam soluções')

In [123]:
text_chunks = [doc.page_content for doc in chunks]
vectorstore = FAISS.from_texts(text_chunks, embedding=embeddings)

In [124]:
vectorstore.save_local("index_faiss")

In [125]:
retriever = vectorstore.as_retriever(search_type = "similarity", search_kwargs={"k": 6})
retriever

VectorStoreRetriever(tags=['FAISS', 'HuggingFaceEmbeddings'], vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x7f1156b57140>, search_kwargs={'k': 6})

In [126]:
prompt_rag = PromptTemplate(
    input_variables=["context", "input"],
    template=template_rag,
)
prompt_rag

PromptTemplate(input_variables=['context', 'input'], input_types={}, partial_variables={}, template='\nPergunta:: {input}\nContexto:: {context}\n')

In [127]:
from langchain_core.runnables import RunnablePassthrough

chain_rag = (
    {"context": retriever, "input": RunnablePassthrough()}
    | prompt_rag
    | llm
    | StrOutputParser()
)

In [130]:
res = chain_rag.invoke("Qual é o publico alvo do banco?")
show(res)

O público-alvo do banco digital é pessoas físicas a partir de 18 anos que buscam soluções bancárias online, com foco em praticidade, segurança e transparência.

In [133]:
#melhoria de prompt

system_prompt = """Você é um assistente virtual prestativo e está respondendo perguntas gerais sobre os serviços de uma empresa.
Use os seguintes pedaços de contexto recuperado para responder à pergunta.
Se você não sabe a resposta, apenas comente que não sabe dizer com certeza.
Mas caso seja uma dúvida muito comum, pode sugerir como alternativa uma solução possível.
Mantenha a resposta concisa.
Responda em português. \n\n"""

qa_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system_prompt),
        ("human", "Pergunta: {input}\n\n Contexto: {context}"),
    ]
)
qa_prompt

ChatPromptTemplate(input_variables=['context', 'input'], input_types={}, partial_variables={}, messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], input_types={}, partial_variables={}, template='Você é um assistente virtual prestativo e está respondendo perguntas gerais sobre os serviços de uma empresa.\nUse os seguintes pedaços de contexto recuperado para responder à pergunta.\nSe você não sabe a resposta, apenas comente que não sabe dizer com certeza.\nMas caso seja uma dúvida muito comum, pode sugerir como alternativa uma solução possível.\nMantenha a resposta concisa.\nResponda em português. \n\n'), additional_kwargs={}), HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context', 'input'], input_types={}, partial_variables={}, template='Pergunta: {input}\n\n Contexto: {context}'), additional_kwargs={})])

In [134]:
chain_rag = (
    {"context": retriever, "input": RunnablePassthrough()}
    | qa_prompt
    | llm
    | StrOutputParser()
)

res = chain_rag.invoke("Como alterar minha senha?")
show(res)

Para alterar sua senha, você pode seguir os passos abaixo:

- No aplicativo: Clique no menu "Minha conta" e selecione "Alterar senha".
- No site: Acesse "Configurações" no menu do topo, selecione "Minha conta" e "Alterar senha".

O sistema exigirá a senha atual e, por segurança, não permitirá reutilizar senhas anteriores recentes.

In [135]:
#Melhorias no metodo de busca
retriever = vectorstore.as_retriever(
    search_type='mmr',
    search_kwargs={'k':3, 'fetch_k':4}
)

#reformulação de consulta para contextualizar

In [138]:
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.messages import AIMessage, HumanMessage

from langchain_classic.chains.history_aware_retriever import create_history_aware_retriever
from langchain_classic.chains.retrieval import create_retrieval_chain
from langchain_classic.chains.combine_documents import create_stuff_documents_chain



context_q_system_prompt = "Given the following chat history and the follow-up question which might reference context in the chat history, formulate a standalone question which can be understood without the chat history. Do NOT answer the question, just reformulate it if needed and otherwise return it as is."
context_q_system_prompt = context_q_system_prompt
context_q_user_prompt = "Question: {input}"
context_q_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", context_q_system_prompt),
        MessagesPlaceholder("chat_history"),
        ("human", context_q_user_prompt),
    ]
)

In [141]:
#chain de contextualização
history_aware_retriever = create_history_aware_retriever(
    llm = llm, retriever = retriever, prompt = context_q_prompt
)

In [142]:
#chain para perguntas e respostas Q&A
qa_prompt = ChatPromptTemplate.from_messages([
    ("system", system_prompt),
    MessagesPlaceholder("chat_history"),
    ("human", "Pergunta: {input}\n\n Contexto: {context}"),
])

qa_chain = create_stuff_documents_chain(llm, qa_prompt)

rag_chain = create_retrieval_chain(history_aware_retriever, qa_chain)

In [144]:
from langchain_core.prompts import MessagesPlaceholder
from langchain_core.messages import AIMessage, HumanMessage
from langchain_classic.chains import create_history_aware_retriever, create_retrieval_chain
from langchain_classic.chains.combine_documents import create_stuff_documents_chain

def config_rag_chain(llm, retriever):

    # Prompt de contextualização
    context_q_system_prompt = "Given the following chat history and the follow-up question which might reference context in the chat history, formulate a standalone question which can be understood without the chat history. Do NOT answer the question, just reformulate it if needed and otherwise return it as is."

    context_q_system_prompt = context_q_system_prompt
    context_q_user_prompt = "Question: {input}"
    context_q_prompt = ChatPromptTemplate.from_messages(
        [
            ("system", context_q_system_prompt),
            MessagesPlaceholder("chat_history"),
            ("human", context_q_user_prompt),
        ]
    )

    # Chain para contextualização
    history_aware_retriever = create_history_aware_retriever(
        llm=llm, retriever=retriever, prompt=context_q_prompt
    )

    # Prompt para perguntas e respostas (Q&A)
    system_prompt = """Você é um assistente virtual prestativo e está respondendo perguntas gerais sobre os serviços de uma empresa.
    Use os seguintes pedaços de contexto recuperado para responder à pergunta.
    Se você não sabe a resposta, apenas comente que não sabe dizer com certeza.
    Mas caso seja uma dúvida muito comum, pode sugerir como alternativa uma solução possível.
    Mantenha a resposta concisa.
    Responda em português. \n\n"""

    qa_prompt = ChatPromptTemplate.from_messages([
        ("system", system_prompt),
        MessagesPlaceholder("chat_history"),
        ("human", "Pergunta: {input}\n\n Contexto: {context}"),
    ])

    # Configurar LLM e Chain para perguntas e respostas (Q&A)

    qa_chain = create_stuff_documents_chain(llm, qa_prompt)

    rag_chain = create_retrieval_chain(
        history_aware_retriever,
        qa_chain,
    )

    return rag_chain

#Historico de conversas

In [145]:
store = {}

In [147]:
from langchain_core.runnables.history import RunnableWithMessageHistory
from langchain_community.chat_message_histories import ChatMessageHistory
from langchain_core.chat_history import BaseChatMessageHistory

def get_session_history(session_id: str) -> BaseChatMessageHistory:
  if session_id not in store:
    store[session_id] = ChatMessageHistory()
  return store[session_id]


In [148]:
qa_chain_with_history = RunnableWithMessageHistory(
    qa_chain,
    get_session_history,
    input_messages_key = "input",
    history_messages_key = "chat_history"
)

In [149]:
rag_chain = config_rag_chain(llm, retriever)

In [150]:
chat_history = []

In [151]:
input = "como alterar minha senha?"

# 1
chat_history.append(HumanMessage(content=input))

# 2
result = rag_chain.invoke({"input": input, "chat_history": chat_history})

res = result['answer']
print(res)

# 3
chat_history.append(AIMessage(content=res))

Para alterar sua senha, você pode seguir os passos abaixo:

- No aplicativo: Clique no menu 'Minha conta' e selecione 'Alterar senha'.
- No site: Acesse 'Configurações' no menu do topo e siga as instruções.

Caso esqueça sua senha, clique em "Esqueci minha senha" na tela de login, informe o CPF e siga as instruções enviadas por e-mail ou SMS para criar uma nova senha.


In [152]:
chat_history

[HumanMessage(content='como alterar minha senha?', additional_kwargs={}, response_metadata={}),
 AIMessage(content='Para alterar sua senha, você pode seguir os passos abaixo:\n\n- No aplicativo: Clique no menu \'Minha conta\' e selecione \'Alterar senha\'.\n- No site: Acesse \'Configurações\' no menu do topo e siga as instruções.\n\nCaso esqueça sua senha, clique em "Esqueci minha senha" na tela de login, informe o CPF e siga as instruções enviadas por e-mail ou SMS para criar uma nova senha.', additional_kwargs={}, response_metadata={})]

In [153]:
input = "Qual foi minha última pergunta?"
result = rag_chain.invoke({
    "input": input,
    "chat_history": chat_history
})
res = result["answer"]
print(res)

Sua última pergunta foi sobre como alterar a senha.


#Configuração a função de Indexação e Recuperação

In [157]:
from pathlib import Path
from langchain_community.document_loaders import PyMuPDFLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_community.vectorstores import FAISS


def config_retriever(folder_path="/content"):
    # Carregar documentos
    docs_path = Path("/content")
    pdf_files = [f for f in docs_path.glob("*.pdf")]

    loaded_documents = [extract_text_pdf(pdf) for pdf in pdf_files]

    # Divisão em pedaços de texto / Split
    text_splitter = RecursiveCharacterTextSplitter(
        chunk_size=1000,
        chunk_overlap=200
    )
    chunks = []
    for doc in loaded_documents:
        chunks.extend(text_splitter.split_text(doc))

    # Embeddings
    embedding_model = "BAAI/bge-m3"

    embeddings = HuggingFaceEmbeddings(model_name=embedding_model)

    # Armazenamento
    vectorstore = FAISS.from_texts(chunks, embedding=embeddings)

    vectorstore.save_local('index_faiss')

    # Configurando o recuperador de texto / Retriever
    retriever = vectorstore.as_retriever(
        search_type='mmr',
        search_kwargs={'k':3, 'fetch_k':4}
    )

    return retriever

In [158]:
retriever = config_retriever()

In [159]:
rag_chain = config_rag_chain(llm, retriever)

#pipeline final + Conclusão

In [160]:
def chat_llm(rag_chain, input, chat_history):
  chat_history.append(HumanMessage(content = input))

  response = rag_chain.invoke({
      "input": input,
      "chat_history": chat_history
  })

  res = response["answer"]

  chat_history.append(AIMessage(content = res))

  return chat_history

In [161]:
chat_history = []

In [162]:
input = "olá! como mudar minha senha?"  # @param {type:"string"}

chat_history = chat_llm(rag_chain, input, chat_history)

In [163]:
chat_history

[HumanMessage(content='olá! como mudar minha senha?', additional_kwargs={}, response_metadata={}),
 AIMessage(content='Olá! Para mudar sua senha, você pode seguir os seguintes passos:\n\nNo aplicativo: clique no menu "Minha conta" e selecione "Alterar senha".\n\nNo site: acesse "Configurações" no menu do topo, selecione "Minha conta" e "Alterar senha".\n\nLembre-se de que o sistema exige a senha atual e, por segurança, não permite reutilizar senhas anteriores recentes.', additional_kwargs={}, response_metadata={})]

In [164]:
!pip install -q streamlit python-dotenv
!npm install -q localtunnel

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.0/9.0 MB 49.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 61.3 MB/s eta 0:00:00
⠙⠹⠸⠼⠴⠦⠧⠇⠏⠋⠙⠹⠸⠼⠴⠦⠧⠇⠏⠋⠙⠹⠸⠼
added 22 packages in 3s
⠼
⠼3 packages are looking for funding
⠼  run `npm fund` for details
⠼npm notice
npm notice New major version of npm available! 10.8.2 -> 11.7.0
npm notice Changelog: https://github.com/npm/cli/releases/tag/v11.7.0
npm notice To update run: npm install -g npm@11.7.0
npm notice
⠼

In [176]:
%%writefile .env
GROQ_API_KEY=

Overwriting .env


In [174]:
%%writefile app02.py
import streamlit as st
from langchain_groq import ChatGroq
from langchain_core.prompts import ChatPromptTemplate
from pathlib import Path
from langchain_community.document_loaders import PyMuPDFLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_core.prompts import MessagesPlaceholder
from langchain_core.messages import AIMessage, HumanMessage
from langchain_classic.chains import create_history_aware_retriever, create_retrieval_chain
from langchain_classic.chains.combine_documents import create_stuff_documents_chain
from langchain_community.vectorstores import FAISS
from dotenv import load_dotenv
load_dotenv()

st.set_page_config(page_title="Atendimento SafeBank 🤖", page_icon="🤖")
st.title("Atendimento SafeBank")

id_model = "llama-3.3-70b-versatile"
temperature = 0.7
path = "/content"

### Carregamento da LLM
def load_llm(id_model, temperature):
  llm = ChatGroq(
    model=id_model,
    temperature=temperature,
    max_tokens=None,
    timeout=None,
    max_retries=2,
  )
  return llm

llm = load_llm(id_model, temperature)

### Exibição do resultado
def show_res(res):
  from IPython.display import Markdown
  if "</think>" in res:
    res = res.split("</think>")[-1].strip()
  else:
    res = res.strip()  # fallback se não houver tag
  display(Markdown(res))

### Extração do conteúdo
def extract_text_pdf(file_path):
  loader = PyMuPDFLoader(file_path)
  doc = loader.load()
  content = "\n".join([page.page_content for page in doc])
  return content

### Indexação e recuperação
def config_retriever(folder_path="/content"):
  # Carregar documentos
  docs_path = Path("/content")
  pdf_files = [f for f in docs_path.glob("*.pdf")]

  if len(pdf_files) < 1:
    st.error("Nenhum arquivo PDF carregado")
    st.stop()

  loaded_documents = [extract_text_pdf(pdf) for pdf in pdf_files]

  # Divisão em pedaços de texto / Split
  text_splitter = RecursiveCharacterTextSplitter(
      chunk_size=1000,
      chunk_overlap=200
  )
  chunks = []
  for doc in loaded_documents:
      chunks.extend(text_splitter.split_text(doc))

  # Embeddings
  embedding_model = "BAAI/bge-m3" #sentence-transformers/all-mpnet-base-v2

  embeddings = HuggingFaceEmbeddings(model_name=embedding_model)

  # Armazenamento
  vectorstore = FAISS.from_texts(chunks, embedding=embeddings)

  vectorstore.save_local('index_faiss')

  # Configurando o recuperador de texto / Retriever
  retriever = vectorstore.as_retriever(
      search_type='mmr',
      search_kwargs={'k':3, 'fetch_k':4}
  )

  return retriever

### Chain da RAG
def config_rag_chain(llm, retriever):

  # Prompt de contextualização
  context_q_system_prompt = "Given the following chat history and the follow-up question which might reference context in the chat history, formulate a standalone question which can be understood without the chat history. Do NOT answer the question, just reformulate it if needed and otherwise return it as is."

  context_q_system_prompt = context_q_system_prompt
  context_q_user_prompt = "Question: {input}"
  context_q_prompt = ChatPromptTemplate.from_messages(
      [
          ("system", context_q_system_prompt),
          MessagesPlaceholder("chat_history"),
          ("human", context_q_user_prompt),
      ]
  )

  # Chain para contextualização
  history_aware_retriever = create_history_aware_retriever(
    llm=llm, retriever=retriever, prompt=context_q_prompt
  )

  # Prompt para perguntas e respostas (Q&A)
  system_prompt = """Você é um assistente virtual prestativo e está respondendo perguntas gerais sobre os serviços de uma empresa.
  Use os seguintes pedaços de contexto recuperado para responder à pergunta.
  Se você não sabe a resposta, apenas comente que não sabe dizer com certeza.
  Mas caso seja uma dúvida muito comum, pode sugerir como alternativa uma solução possível.
  Mantenha a resposta concisa.
  Responda em português. \n\n"""

  qa_prompt = ChatPromptTemplate.from_messages([
    ("system", system_prompt),
    MessagesPlaceholder("chat_history"),
    ("human", "Pergunta: {input}\n\n Contexto: {context}"),
  ])

  # Configurar LLM e Chain para perguntas e respostas (Q&A)

  qa_chain = create_stuff_documents_chain(llm, qa_prompt)

  rag_chain = create_retrieval_chain(
    history_aware_retriever,
    qa_chain,
  )

  return rag_chain

### Interação com chat
def chat_llm(rag_chain, input):

  st.session_state.chat_history.append(HumanMessage(content=input))

  response = rag_chain.invoke({
      "input": input,
      "chat_history": st.session_state.chat_history
  })

  res = response["answer"]
  res = res.split("</think>")[-1].strip() if "</think>" in res else res.strip()

  st.session_state.chat_history.append(AIMessage(content=res))

  return res

input = st.chat_input("Digite sua mensagem aqui...")

if "chat_history" not in st.session_state:
  st.session_state.chat_history = [
      AIMessage(content = "Olá, sou o seu assistente virtual! Como posso te ajudar?"),
  ]

if "retriever" not in st.session_state:
  st.session_state.retriever = None

for message in st.session_state.chat_history:
  if isinstance(message, AIMessage):
    with st.chat_message("AI"):
      st.write(message.content)
  elif isinstance(message, HumanMessage):
    with st.chat_message("Human"):
      st.write(message.content)

if input is not None:
  with st.chat_message("Human"):
    st.markdown(input)

  with st.chat_message("AI"):
    if st.session_state.retriever is None:
      st.session_state.retriever = config_retriever(path)
    rag_chain = config_rag_chain(llm, st.session_state.retriever)
    res = chat_llm(rag_chain, input)
    st.write(res)

Overwriting app02.py


In [170]:
!pip install pyngrok

In [175]:
from pyngrok import ngrok


!ngrok config add-authtoken
!streamlit run app02.py --server.port 8501 &>/content/logs.txt &

public_url = ngrok.connect(8501)
public_url

Authtoken saved to configuration file: /root/.config/ngrok/ngrok.yml


<NgrokTunnel: "https://stephanie-nondisciplinary-bonnie.ngrok-free.dev" -> "http://localhost:8501">